In [8]:
import os
import sys
import platform
sys.path.append(os.path.join(os.getcwd().replace("notebooks/pruning_quantization", "")))

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from config import Config
from data.dataset import NetworkDataset, load_datasets
from model_config import MLP_Models, LSTM_Models, GRU_Models, CNN_models
from utils.benchmark import Benchmark
from compact.quantization import quantization_fx, dynamic_quantize
from utils.plot import plot_fpr_tpr_roc_auc

In [9]:
conf = Config()
load_model = CNN_models()
model_conf = load_model.cnn_pq
model = load_model.get_model(model_conf)
model.load()
print(conf.device)

Checkpoint loaded from /global/D1/homes/jorgetf/Network-Packet-ML-Model/checkpoint/pruned_quantized/cnn_pq.pth!
cpu


In [10]:
# data loader
X_train, y_train, X_val, y_val, X_test, y_test = load_datasets(conf.datasets, model_type=load_model.type)
X_test, y_test = X_test[:500*conf.batch_size], y_test[:500*conf.batch_size]
print(X_test.shape)

dataset = NetworkDataset(X_test, y_test)
loader = DataLoader(dataset, conf.batch_size, shuffle=True)

torch.Size([500, 1, 513])


In [11]:
data, label = next(iter(loader))
print(data.shape, label.shape)

torch.Size([1, 1, 513]) torch.Size([1])


In [12]:
# quantize model
name = model_conf["name"]
result_path = os.path.join(conf.benchmark_host, "pruned_quantized_model", "quantized_" + name + ".txt")
model.load()

quantized_model = dynamic_quantize(model.model)
#quantized_model = quantization_fx(model.model, loader, load_model.type)
model.model = quantized_model

Checkpoint loaded from /global/D1/homes/jorgetf/Network-Packet-ML-Model/checkpoint/pruned_quantized/cnn_pq.pth!


In [13]:
# benchmark model
benchmark = Benchmark(model, loader, conf.batch_size, name, result_path)
benchmark.metrics()

# print and save result
benchmark.print_result()
benchmark.save()

tensor([ 0,  0,  0, 14, 10,  5, 10,  0,  0,  0])
tensor([205, 214,  78,  61,  57,  46,  57, 174, 232, 117])


/global/D1/homes/jorgetf/Network-Packet-ML-Model/.venv/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1201: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/global/D1/homes/jorgetf/Network-Packet-ML-Model/.venv/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1201: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/global/D1/homes/jorgetf/Network-Packet-ML-Model/.venv/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1201: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/global/D1/homes/jorgetf/Network-Packet-ML-Model/.venv/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1201: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/global/D1/homes/jorgetf/Network-Packet-ML-Model/.venv/lib/python3.10/si

Benchmark - cnn_pq model:

Model (cnn_pq) Macro-F1, Micro-F1 and Macro ROC AUC scores:
Macro-F1 score: 0.00
Micro-F1 score: 0.00
Macro ROC AUC score: nan





/global/D1/homes/jorgetf/Network-Packet-ML-Model/.venv/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1201: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/global/D1/homes/jorgetf/Network-Packet-ML-Model/.venv/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1201: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/global/D1/homes/jorgetf/Network-Packet-ML-Model/.venv/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1201: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/global/D1/homes/jorgetf/Network-Packet-ML-Model/.venv/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1201: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/global/D1/homes/jorgetf/Network-Packet-ML-Model/.venv/lib/python3.10/si

In [14]:
#loss, metrics = model.evaluate(loader)
#print(f"Macro-F1 score: {metrics['f1_macro']:.2f}, Micro-F1 score: {metrics['f1_micro']:.2f}, Macro ROC AUC score: {metrics['roc_auc_macro']:.2f}")